In [16]:
import pandas as pd

## Étape 1 : Charger les deux fichiers de salariés

In [17]:
# Charger le premier fichier
df1 = pd.read_csv('merged_salaries.csv')
print("=== FICHIER 1: merged_salaries.csv ===")
print(f"Nombre de lignes : {len(df1)}")
print(f"Nombre de colonnes : {len(df1.columns)}")
print(f"Colonnes : {list(df1.columns)}")
print()
df1.head()

=== FICHIER 1: merged_salaries.csv ===
Nombre de lignes : 858623
Nombre de colonnes : 6
Colonnes : ['ID_adherent', 'month', 'immatriculationNumber', 'full_name', 'nombre_jours', 'salaire']



,ID_adherent,month,immatriculationNumber,full_name,nombre_jours,salaire
0,10003,décembre 2023,103250842,Meniar Mohammed,26,22597.56
1,10003,décembre 2023,103251048,Boujrhoul Driss,26,30067.30
2,10003,décembre 2023,103251147,Afkare Abdelkrim,26,32787.71
3,10003,décembre 2023,103251246,Bouzidi Mohammed,26,26087.30
4,10003,décembre 2023,103251345,Driouich Mostapha,22,23936.21


In [18]:
# Charger le deuxième fichier
df2 = pd.read_csv('dataset_encoded_salaries.csv')
print("=== FICHIER 2: dataset_encoded_salaries.csv ===")
print(f"Nombre de lignes : {len(df2)}")
print(f"Nombre de colonnes : {len(df2.columns)}")
print(f"Colonnes : {list(df2.columns)}")
print()
df2.head()

=== FICHIER 2: dataset_encoded_salaries.csv ===
Nombre de lignes : 1994090
Nombre de colonnes : 5
Colonnes : ['ID_adherent', 'immatriculationNumber', 'demandMode', 'affiliateNumber', 'anciennete_adhesion_months']



,ID_adherent,immatriculationNumber,demandMode,affiliateNumber,anciennete_adhesion_months
0,67,143527311,0,1905845.0,82.0
1,67,143527412,0,1905845.0,82.0
2,67,144415719,0,1905845.0,82.0
3,67,144415810,0,1905845.0,82.0
4,67,144415911,0,1905845.0,82.0


## Étape 2 : Analyser les colonnes communes et différentes

In [19]:
# Identifier les colonnes communes et différentes
cols_df1 = set(df1.columns)
cols_df2 = set(df2.columns)

colonnes_communes = cols_df1 & cols_df2
colonnes_uniques_df1 = cols_df1 - cols_df2
colonnes_uniques_df2 = cols_df2 - cols_df1

print("=== ANALYSE DES COLONNES ===\n")
print(f"Colonnes communes aux deux fichiers ({len(colonnes_communes)}):")
print(list(colonnes_communes))
print()
print(f"Colonnes uniques à merged_salaries.csv ({len(colonnes_uniques_df1)}):")
print(list(colonnes_uniques_df1))
print()
print(f"Colonnes uniques à dataset_encoded_salaries.csv ({len(colonnes_uniques_df2)}):")
print(list(colonnes_uniques_df2))

=== ANALYSE DES COLONNES ===

Colonnes communes aux deux fichiers (2):
['ID_adherent', 'immatriculationNumber']

Colonnes uniques à merged_salaries.csv (4):
['nombre_jours', 'month', 'full_name', 'salaire']

Colonnes uniques à dataset_encoded_salaries.csv (3):
['anciennete_adhesion_months', 'demandMode', 'affiliateNumber']


## Étape 3 : Fusionner les deux fichiers sur immatriculationNumber

In [20]:
# Fusionner les deux datasets sur immatriculationNumber (INNER JOIN)
# Cela garde seulement les salariés présents dans les DEUX fichiers
dataset_combined = pd.merge(
    df1, 
    df2, 
    on='immatriculationNumber', 
    how='inner',  # Garde seulement les correspondances
    suffixes=('_file1', '_file2')  # Suffixes pour les colonnes en double
)

print("=== RÉSULTAT DE LA FUSION ===")
print(f"Lignes dans fichier 1 : {len(df1)}")
print(f"Lignes dans fichier 2 : {len(df2)}")
print(f"Lignes après fusion : {len(dataset_combined)}")
print(f"Colonnes totales : {len(dataset_combined.columns)}")
print()
print("Colonnes du dataset fusionné :")
print(list(dataset_combined.columns))

=== RÉSULTAT DE LA FUSION ===
Lignes dans fichier 1 : 858623
Lignes dans fichier 2 : 1994090
Lignes après fusion : 195594
Colonnes totales : 10

Colonnes du dataset fusionné :
['ID_adherent_file1', 'month', 'immatriculationNumber', 'full_name', 'nombre_jours', 'salaire', 'ID_adherent_file2', 'demandMode', 'affiliateNumber', 'anciennete_adhesion_months']


In [21]:
# Afficher un aperçu
dataset_combined.head()

,ID_adherent_file1,month,immatriculationNumber,full_name,nombre_jours,salaire,ID_adherent_file2,demandMode,affiliateNumber,anciennete_adhesion_months
0,10003,décembre 2023,124027116,Chaibi Abdelkader,26,10345.81,321,0,6478235.0,87.0
1,1002,décembre 2023,100038957,Saad Hind,0,0.00,1002,1,1882382.0,34.0
2,1002,décembre 2023,106039614,Ighir Aicha,0,0.00,160000340,0,3561728.0,91.0
3,1002,décembre 2023,109155613,El Araichi Bouchra,0,0.00,60546,0,6518859.0,91.0
4,1002,décembre 2023,109426339,Laaouina Zakaria,0,0.00,1002,1,1882382.0,48.0


## Étape 4 : Analyser les données perdues

In [22]:
# Identifier les salariés qui n'ont pas été conservés
immat_df1 = set(df1['immatriculationNumber'])
immat_df2 = set(df2['immatriculationNumber'])
immat_combined = set(dataset_combined['immatriculationNumber'])

# Salariés dans df1 mais PAS dans df2
perdus_df1 = immat_df1 - immat_df2
# Salariés dans df2 mais PAS dans df1
perdus_df2 = immat_df2 - immat_df1

print("=== ANALYSE DES DONNÉES PERDUES ===\n")
print(f"Salariés dans fichier 1 uniquement : {len(perdus_df1)}")
print(f"Salariés dans fichier 2 uniquement : {len(perdus_df2)}")
print(f"Salariés conservés (présents dans les 2) : {len(immat_combined)}")
print()
print(f"Taux de conservation : {(len(immat_combined) / max(len(immat_df1), len(immat_df2)) * 100):.2f}%")

=== ANALYSE DES DONNÉES PERDUES ===

Salariés dans fichier 1 uniquement : 661281
Salariés dans fichier 2 uniquement : 1798925
Salariés conservés (présents dans les 2) : 194295

Taux de conservation : 9.75%


## Étape 5 : Nettoyer les colonnes dupliquées

In [23]:
# Identifier les colonnes avec suffixes (_file1, _file2)
colonnes_avec_suffixes = [col for col in dataset_combined.columns if '_file1' in col or '_file2' in col]

print(f"Colonnes dupliquées ({len(colonnes_avec_suffixes)}):")
for col in colonnes_avec_suffixes:
    print(f"  - {col}")

Colonnes dupliquées (2):
  - ID_adherent_file1
  - ID_adherent_file2


In [24]:
# Supprimer les colonnes dupliquées (garder seulement _file1 ou choisir la meilleure)
# Option : Supprimer toutes les colonnes _file2 pour éviter la redondance
colonnes_a_supprimer = [col for col in dataset_combined.columns if '_file2' in col]

dataset_clean = dataset_combined.drop(columns=colonnes_a_supprimer)

# Renommer les colonnes _file1 pour enlever le suffixe
dataset_clean.columns = [col.replace('_file1', '') for col in dataset_clean.columns]

print(f"✓ {len(colonnes_a_supprimer)} colonnes dupliquées supprimées")
print(f"\nNombre de colonnes final : {len(dataset_clean.columns)}")
print(f"Colonnes finales : {list(dataset_clean.columns)}")

✓ 1 colonnes dupliquées supprimées

Nombre de colonnes final : 9
Colonnes finales : ['ID_adherent', 'month', 'immatriculationNumber', 'full_name', 'nombre_jours', 'salaire', 'demandMode', 'affiliateNumber', 'anciennete_adhesion_months']


## Étape 6 : Vérifier le dataset final

In [25]:
dataset_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195594 entries, 0 to 195593
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ID_adherent                 195594 non-null  int64  
 1   month                       195594 non-null  object 
 2   immatriculationNumber       195594 non-null  int64  
 3   full_name                   195594 non-null  object 
 4   nombre_jours                195594 non-null  int64  
 5   salaire                     195594 non-null  float64
 6   demandMode                  195594 non-null  int64  
 7   affiliateNumber             195594 non-null  float64
 8   anciennete_adhesion_months  195594 non-null  float64
dtypes: float64(3), int64(4), object(2)
memory usage: 13.4+ MB


In [26]:
dataset_clean.head(10)

,ID_adherent,month,immatriculationNumber,full_name,nombre_jours,salaire,demandMode,affiliateNumber,anciennete_adhesion_months
0,10003,décembre 2023,124027116,Chaibi Abdelkader,26,10345.81,0,6478235.0,87.0
1,1002,décembre 2023,100038957,Saad Hind,0,0.00,1,1882382.0,34.0
2,1002,décembre 2023,106039614,Ighir Aicha,0,0.00,0,3561728.0,91.0
3,1002,décembre 2023,109155613,El Araichi Bouchra,0,0.00,0,6518859.0,91.0
4,1002,décembre 2023,109426339,Laaouina Zakaria,0,0.00,1,1882382.0,48.0
5,1002,décembre 2023,111770623,El Bguira Mohamed,0,0.00,0,3562122.0,68.0
6,1002,décembre 2023,114864827,Jouahri Chayma,0,0.00,0,6518859.0,67.0
7,1002,décembre 2023,115298423,Elfahime Khaddouj,0,0.00,0,6518859.0,67.0
8,1002,décembre 2023,115773229,Chmicha Mouloud,0,0.00,0,6518859.0,67.0
9,1002,décembre 2023,126826112,El Marzouki Rabia,0,0.00,0,3380011.0,86.0


In [27]:
# Colonnes à supprimer
colonnes_a_supprimer = [
    'month', 'full_name', 'ID_adherent', 'immatriculationNumber'
]

# Filtrer seulement les colonnes qui existent dans the dataset
colonnes_existantes = [col for col in colonnes_a_supprimer if col in dataset_clean.columns]

# Supprimer les colonnes
dataset_clean2 = dataset_clean.drop(columns=colonnes_existantes)

print(f"Colonnes supprimées : {colonnes_existantes}")
print(f"\nNombre de colonnes avant : {len(dataset_clean.columns)}")
print(f"Nombre de colonnes après : {len(dataset_clean2.columns)}")
print(f"\nColonnes restantes : {list(dataset_clean2.columns)}")

Colonnes supprimées : ['month', 'full_name', 'ID_adherent', 'immatriculationNumber']

Nombre de colonnes avant : 9
Nombre de colonnes après : 5

Colonnes restantes : ['nombre_jours', 'salaire', 'demandMode', 'affiliateNumber', 'anciennete_adhesion_months']


In [28]:
dataset_clean2['affiliateNumber'] = dataset_clean2['affiliateNumber'].astype(int)
dataset_clean2['anciennete_adhesion_months'] = dataset_clean2['anciennete_adhesion_months'].astype(int)

In [29]:
dataset_clean2.head()

,nombre_jours,salaire,demandMode,affiliateNumber,anciennete_adhesion_months
0,26,10345.81,0,6478235,87
1,0,0.00,1,1882382,34
2,0,0.00,0,3561728,91
3,0,0.00,0,6518859,91
4,0,0.00,1,1882382,48


## Étape 7 : Sauvegarder le dataset combiné

In [30]:
# Afficher l'état avant nettoyage
print("=== AVANT NETTOYAGE ===")
print(f"Nombre de lignes : {len(dataset_clean2)}")
print(f"Nombre de doublons : {dataset_clean2.duplicated().sum()}")
print(f"Nombre de lignes avec valeurs nulles : {dataset_clean2.isnull().any(axis=1).sum()}")
print("\n")

# 1. Supprimer les doublons
dataset_clean2 = dataset_clean2.drop_duplicates()
print(f"Après suppression des doublons : {len(dataset_clean2)} lignes")

# 2. Supprimer les lignes avec valeurs nulles
dataset_clean2 = dataset_clean2.dropna()
print(f"Après suppression des nulles : {len(dataset_clean2)} lignes")
# Afficher l'état après nettoyage
print("\n=== APRÈS NETTOYAGE ===")
print(f"Nombre de lignes final : {len(dataset_clean2)}")
print(f"Valeurs nulles restantes : {dataset_clean2.isnull().sum().sum()}")
print(f"Doublons restants : {dataset_clean2.duplicated().sum()}")

=== AVANT NETTOYAGE ===
Nombre de lignes : 195594
Nombre de doublons : 28022
Nombre de lignes avec valeurs nulles : 0


Après suppression des doublons : 167572 lignes
Après suppression des nulles : 167572 lignes

=== APRÈS NETTOYAGE ===
Nombre de lignes final : 167572
Valeurs nulles restantes : 0
Après suppression des nulles : 167572 lignes

=== APRÈS NETTOYAGE ===
Nombre de lignes final : 167572
Valeurs nulles restantes : 0
Doublons restants : 0
Doublons restants : 0


In [31]:
# Sauvegarder le dataset final
output_file = 'salaries_combined_data.csv'
dataset_clean2.to_csv(output_file, index=False)

print(f"✓ Dataset combiné sauvegardé dans '{output_file}'")
print(f"\n=== RÉSUMÉ FINAL ===")
print(f"Nombre de salariés : {len(dataset_clean2)}")
print(f"Nombre de colonnes : {len(dataset_clean2.columns)}")
print(f"Taille mémoire : {dataset_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

✓ Dataset combiné sauvegardé dans 'salaries_combined_data.csv'

=== RÉSUMÉ FINAL ===
Nombre de salariés : 167572
Nombre de colonnes : 5
Taille mémoire : 36.94 MB
Taille mémoire : 36.94 MB
